### Parallel MPI simulatons of a spherical blast wave

### Spin-Up MPI cluster, compile, and execute simulation code

In [ ]:
# spin up camber
import camber

In [ ]:
# download Athena++ from public repo and compile if needed -- can skip if you have already copied the code to the working directory
# working directory is where the notebook is located
import subprocess
subprocess.run(["git","clone","https://github.com/PrincetonUniversity/athena.git"])

In [ ]:
# Now we create an MPI job to compile Athena++. This step ensure the code is compiled with the correct MPI environment
compile_job = camber.mpi.create_job(
    command="cd athena && python configure.py --prob=blast -mpi -hdf5 --hdf5_path=${HDF5_PATH} && make clean && make all -j$(nproc)",
    engine_size="SMALL"
)

In [ ]:
# we can check the status of the job
compile_job

In [ ]:
# The blast problem begins with a central sphere of material with higher pressure than its surroundings.
# The prat parameter sets the ratio. Here we consider four values ranging from a factor of 10 to a factor of 300.
params = {
    "prat": ["10", "30", "100", "300"]
}

In [ ]:
# run scatter jobs using MEDIUM engines
# Here we take advantage Athena's functionality for over-writing a parameter in the input file frome the
# command line as well as the ability to set the runtime directory
jobs = camber.mpi.create_scatter_job(
    command_template="mpirun -np 16 athena/bin/athena -i athinput.blast problem/prat={prat} -d run{prat}",
    template_params_grid=params,
    engine_size="MEDIUM"
)

In [ ]:
# lets check in on how are jobs are doing
jobs

In [ ]:
# import a custom script for reading and plotting the hdf5 outputs, placing images in the output_images directory
# Each frame is a 2D slice throught the center of the 3D cartesian grid. This assumes each simulation output
# is in its own runtime directory. We pass the same params dictionary used above for the scatter job.
subprocess.run(["mkdir","output_images"])
from plot_output import plot_output
plot_output(params, "prat")

### Visualize Data

In [ ]:
from IPython.display import Video
Video("density.mov")